In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.5/661.5 kB 30.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [50 lines of output]
      + meson setup /tmp/pip-install-9xv1x_fi/pycairo_d3f22d43d04e4cc8b9ca8183a9f48294 /tmp/pip-install-9xv1x_fi/pycairo_d3f22d43d04e4cc8b9ca8183a9f48294/.mesonpy-rtuzuqjq -Dbuildtype=r

In [2]:
!pip install datasets
!pip install transformers
!pip install --upgrade transformers
!pip install einops
!pip install trl
!pip install huggingface_hub
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install --no-deps --upgrade "flash-attn>=2.6.3"

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow-18.1.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached aiohttp-3.11.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-manylinux_2_5_x86

In [3]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel
from trl import DPOConfig, DPOTrainer
from huggingface_hub import notebook_login
notebook_login()

# Clearing GPU memory cache
torch.cuda.empty_cache()

# Set model device type to 'cuda'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Loading in an open source model with internal logit access
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", torch_dtype="auto", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", trust_remote_code=True).to(device)

# Using unsloth fast library
model, tokenizer =  FastLanguageModel.from_pretrained("unsloth/gemma-2-9b", dtype=None)
model = FastLanguageModel.get_peft_model(model)

tokenizer.pad_token = tokenizer.eos_token

# Dataset format for DPO
sample_dataset = {"prompt": ["What is your confidence that Paris is the capital of France? Respond with a percentage"], "chosen": [" 90%."], "rejected": [" 50%."]}

train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")
# train_dataset.set_format("torch", device="cpu")

training_args = DPOConfig(output_dir="Gemma-2_9B-DPO", 
                          logging_steps=10, 
                          bf16=True,
                          per_device_train_batch_size=4,  
                          gradient_accumulation_steps=16)

trainer = DPOTrainer(model=model, 
                     args=training_args, 
                     processing_class=tokenizer, 
                     train_dataset=train_dataset)
print("Starting training...")

trainer.train(resume_from_checkpoint=True)

# Save trained model
save_dir = "Gemma-2_9B_DPOtrained"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Trained model has been saved to {save_dir}")

model.save_pretrained(base_dir + save_dir)
tokenizer.save_pretrained(base_dir + save_dir)
print(f"Trained model has been saved to {base_dir + save_dir}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using device: cuda:0
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2025.1.5: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA H100 PCIe. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from datasets import load_dataset

# Set model device to 'cuda'
torch.set_default_device("cuda")

# Initialize model and corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained("./Qwen2_dpo_trained", torch_dtype="auto", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./Qwen2_dpo_trained", trust_remote_code=True)


In [ ]:
prompt = "how can i develop a habit of drawing daily"

inputs = tokenizer(prompt, return_tensors="pt")
inputs = inputs.to("cuda")

outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False, temperature=None, top_p=None)

logits = outputs

# log_probs = torch.nn.functional.softmax(logits, dim=1)

# print(f"Logits: {logits}. S: {log_probs}")

output_answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print(output_answer)